## Setup

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
from supabase import create_client, Client
import pandas as pd
import dotenv
import os
import uuid
from datetime import datetime, timezone


SERVICE_ACCOUNT_KEY = "./../serviceAccountKey.json"
FILE_PATH = "./../Documentation\SETS Team, RS, District Officers - SETS Dubar 2025 (Responses).xlsx"

dotenv.load_dotenv()

True

In [2]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred)
db = firestore.client()
supabase = create_client(os.getenv('EXPO_PUBLIC_SUPABASE_URL'), os.getenv('EXPO_PUBLIC_SUPABASE_ANON_KEY'))

In [37]:
df = pd.read_excel(FILE_PATH)

## Preprocess

In [38]:
df

,Timestamp,Email address,Full Name,Affiliation,Designation,Date of Birth,Sex,Photograph,Spouse's Name,Wedding Anniversary,...,Type of Business,Business Address,Business Website,About your business,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Phone,T Shirt Size,Shirt Size,Meal Preference
0,2024-12-19 10:03:04.673,gopi@fin1solutions.com,R GOPINATH,SETS Team,SETS SECRETARY,1972-02-25,Male,https://drive.google.com/open?id=18w_j4W8cQtuI...,VIJAYALAKSHMI,1998-11-02,...,Financial Services,NO 264 B BLOCK FLAT NO 4 ANBUR COLONY ANNA NAG...,www.fin1solutions.com,Fin 1 Solutions is a DSA for HDFC Bank for the...,TARUN GOPINATH,SON,7358543242,L,42,Non-Vegetarian
1,2024-12-26 19:22:13.109,rtn.uma1505@gmail.com,Rtn. Uma Yuvaraj,SETS Team,Ex AG,1980-05-15,Female,https://drive.google.com/open?id=1_zb_c7fblsNC...,Rtn. Yuvaraj,2020-03-24,...,Health and Wellness,.,notapplicable.com,..,Yuvaraj,Spouse,9361455621,M,0,Vegetarian
2,2024-12-26 19:35:50.786,advocatesuresh763@gmail.com,Sureshselvakumar,SETS Team,President,1965-05-31,Male,https://drive.google.com/open?id=1Yrt2omdhAWWu...,Subbulakshmi,1965-05-31,...,Legal services,"FOM, 4 th floor, Jain citadel Apartment, Justi...",NaN,"My father is retired Judge, I handle all type ...",Subbulakshmi,Wife,9790708712,XXXL,48,Non-Vegetarian
3,2024-12-26 19:38:22.354,nokiasai@gmail.com,M Sesha Sai,SETS Team,DRCC,1965-08-22,Male,https://drive.google.com/open?id=1dK5I2dsGoxeW...,M Kamala,1989-05-19,...,Agriculture and Farming,62 Veerabhadran street. Nungambakkam. Chennai ...,www.farmguru.org,Farm Guru is a social initiative that educates...,M Kamala,Wife,9840546655,L,42,Non-Vegetarian
4,2024-12-27 11:10:49.283,bharatmpujara@gmail.com,Bharat Pujara,SETS Team,Chairman,1955-04-15,Male,https://drive.google.com/open?id=1wZj4WhcrnMn1...,Bharati pujara,1976-02-07,...,Paper Recycling,6/7 5th street T K salai Mylapore Chennai 600004,NaN,Suppling Raw material to paper mills for Recyc...,Bharati Pujara,Wife,9840938880,XXXL,50,Vegetarian
5,2024-12-27 12:55:52.213,sivailangovan@gmail.com,Siva Ilangovan,SETS Team,Co Chairman,1967-06-10,Male,https://drive.google.com/open?id=1HtZeulncNUBn...,Geetha,1996-05-19,...,Industrial Air conditioning - HVAC Cleanroom s...,"C1, Swathy Nest, 16/3, Poonamallee High Road, ...",NaN,"Design, construction and contracting of HVAC s...",Geetha,Spouse,9840166223,XL,42,Non-Vegetarian


In [39]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_').str.replace("'", "")
df.rename(columns={
    "if_self-employed_:_business_name_/_if_salaried_:_company_name": "company_name",
    "designation": "sets_designation",
    "role_in_business_/_designation_in_company": "designation",
    "full_name": "name"}, inplace=True)
df.drop(columns=['timestamp'], inplace=True)

In [40]:
df["name_lower"] = df["name"].str.lower()
df["company_name_lower"] = df["company_name"].str.lower()
df["club_name_lower"] = df["club_name"].str.lower()
df["priority"] = 0
df['role'] = "member"
df["support"] = True
df.fillna("", inplace=True)

In [41]:
def convert_timestamp_to_string(timestamp):
    if isinstance(timestamp, datetime):
        if timestamp.tzinfo is None:  # Check if the timestamp is naive
            timestamp = timestamp.replace(tzinfo=timezone.utc)
        else:
            timestamp = timestamp.astimezone(timezone.utc)
    formatted_string = timestamp.strftime('%Y-%m-%d %H:%M:%S%z')
    if formatted_string.endswith("+0000"):
        formatted_string = formatted_string[:-2] + ":00"
    return formatted_string

In [42]:
df.columns

Index(['email_address', 'name', 'affiliation', 'sets_designation',
       'date_of_birth', 'sex', 'photograph', 'spouses_name',
       'wedding_anniversary', 'phone', 'email', 'club_name', 'rotarian_since',
       'rotary_foundation_title', 'residential_address', 'company_name',
       'designation', 'type_of_business', 'business_address',
       'business_website', 'about_your_business', 'emergency_contact_name',
       'emergency_contact_relationship', 'emergency_contact_phone',
       't_shirt_size', 'shirt_size', 'meal_preference', 'name_lower',
       'company_name_lower', 'club_name_lower', 'priority', 'role', 'support'],
      dtype='object')

In [43]:
datetime_columns = df.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns

In [44]:
datetime_columns

Index(['date_of_birth', 'wedding_anniversary'], dtype='object')

In [45]:
for col in datetime_columns:
    df[col] = df[col].dt.tz_localize("UTC", ambiguous="NaT", nonexistent="NaT")

In [46]:
for col in datetime_columns:
    df[col] = df[col].apply(convert_timestamp_to_string)

In [47]:
data = df.to_dict(orient='records')

## Upsert Users

- Check for email existence in firebase auth, if it exists then update record in firestore
- If email does not exist then create new user in firebase auth and then create new record in firestore with doc id = user.uid

In [50]:
for user_data in data:
    email = user_data.get("email")
    if not email:
        print("Skipping record without email:", user_data)
        continue
    
    try:
        # Check if user exists in Firebase Auth
        user = auth.get_user_by_email(email)
        print(f"User exists: {email} (UID: {user.uid}). Updating record")
        
        # Update Firestore record
        user_data['id'] = user.uid
        try:
            # Upsert the record into Supabase
            response = supabase.table("members").upsert(user_data).execute()

            if response.error:
                print(f"Error while upserting record for email {email}: {response.error.message}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")
        
    except auth.UserNotFoundError:
        print(f"User does not exist: {email}")
        
        # Create new user in Firebase Auth
        new_user = auth.create_user(email=email, password=email.split('@')[0] + '_password')
        print(f"Created new user: {email} (UID: {new_user.uid})")
        
        # Create new Firestore record
        user_data['id'] = user.uid
        try:
        # Upsert the record into Supabase
            response = supabase.table("members").upsert(user_data).execute()

            if response.error:
                print(f"Error while upserting record for email {email}: {response.error.message}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")

User exists: gopi@fin1solutions.com (UID: MnEZWLNoTqXsSgvUSlH3Sr4I0A72). Updating record
An error occurred while processing email gopi@fin1solutions.com: 'APIResponse[~_ReturnT]' object has no attribute 'error'
User exists: rtn.uma1505@gmail.com (UID: E3kekKeFkTNDdp6rgG6wtgFYmWY2). Updating record
An error occurred while processing email rtn.uma1505@gmail.com: 'APIResponse[~_ReturnT]' object has no attribute 'error'
User exists: advocatesuresh763@yahoo.com (UID: 2DXGEuXDgdXBAAzRgnyLhCghD8D2). Updating record
An error occurred while processing email advocatesuresh763@yahoo.com: 'APIResponse[~_ReturnT]' object has no attribute 'error'
User exists: rotaryclubsai@gmail.com (UID: Pntjifnvm5TmFVWpLvRRS5JRTEn1). Updating record
An error occurred while processing email rotaryclubsai@gmail.com: 'APIResponse[~_ReturnT]' object has no attribute 'error'
User exists: bharatmpujara@gmail.com (UID: 6huDCteYDyfr2OXenMVVEQjMufs2). Updating record
An error occurred while processing email bharatmpujara@g

In [3]:
records = [
    {
        "user_id": "EU1O3wRX8UfVa7fs0CpYev9eBsc2",
        "itinerary_id": "eIO1lSRwFdmgCtvOC3PX",
        "comment": "A very informative session with practical tips for new and experienced secretaries alike."
    },
    {
        "user_id": "wYCW25rYsbRqtANt7gAc7vjqNxh2",
        "itinerary_id": "eIO1lSRwFdmgCtvOC3PX",
        "comment": "The training provided great insight into the responsibilities of a Rotary secretary."
    },
    {
        "user_id": "6huDCteYDyfr2OXenMVVEQjMufs2",
        "itinerary_id": "eIO1lSRwFdmgCtvOC3PX",
        "comment": "I learned a lot about the importance of communication and organization within the club."
    },
    {
        "user_id": "EU1O3wRX8UfVa7fs0CpYev9eBsc2",
        "itinerary_id": "6AsZlajvoMCNd3M46M7c",
        "comment": "The seminar covered everything I needed to know about managing club records and meetings."
    },
    {
        "user_id": "wYCW25rYsbRqtANt7gAc7vjqNxh2",
        "itinerary_id": "6AsZlajvoMCNd3M46M7c",
        "comment": "It was a valuable experience— I now feel much more confident in my role as a Rotary secretary."
    },
    {
        "user_id": "6huDCteYDyfr2OXenMVVEQjMufs2",
        "itinerary_id": "6AsZlajvoMCNd3M46M7c",
        "comment": "Great content and helpful resources to guide me through the administrative duties of the club."
    },
    {
        "user_id": "EU1O3wRX8UfVa7fs0CpYev9eBsc2",
        "itinerary_id": "G6GVSJRWuz4R21R68h4R",
        "comment": "The speakers were knowledgeable and provided clear, actionable advice."
    },
    {
        "user_id": "wYCW25rYsbRqtANt7gAc7vjqNxh2",
        "itinerary_id": "G6GVSJRWuz4R21R68h4R",
        "comment": "I appreciated the opportunity to connect with other Rotary secretaries and share best practices."
    },
    {
        "user_id": "6huDCteYDyfr2OXenMVVEQjMufs2",
        "itinerary_id": "G6GVSJRWuz4R21R68h4R",
        "comment": "The session on club reporting and recordkeeping was especially useful."
    },
    {
        "user_id": "EU1O3wRX8UfVa7fs0CpYev9eBsc2",
        "itinerary_id": "MJCMsw4ajERwm2qmNzvD",
        "comment": "Excellent overview of the Rotary Secretary role, from handling correspondence to managing agendas."
    },
    {
        "user_id": "wYCW25rYsbRqtANt7gAc7vjqNxh2",
        "itinerary_id": "MJCMsw4ajERwm2qmNzvD",
        "comment": "The hands-on exercises gave me a better understanding of the tools available for managing club operations."
    },
    {
        "user_id": "6huDCteYDyfr2OXenMVVEQjMufs2",
        "itinerary_id": "MJCMsw4ajERwm2qmNzvD",
        "comment": "I found the tips on effective meeting minutes and communication particularly helpful."
    }
]


In [4]:
supabase.table("feedback").upsert(records).execute()

APIResponse[~_ReturnT](data=[{'id': 27, 'comment': 'A very informative session with practical tips for new and experienced secretaries alike.', 'itinerary_id': 'eIO1lSRwFdmgCtvOC3PX', 'user_id': 'EU1O3wRX8UfVa7fs0CpYev9eBsc2', 'created_at': '2024-12-31T07:11:21.127422+00:00'}, {'id': 28, 'comment': 'The training provided great insight into the responsibilities of a Rotary secretary.', 'itinerary_id': 'eIO1lSRwFdmgCtvOC3PX', 'user_id': 'wYCW25rYsbRqtANt7gAc7vjqNxh2', 'created_at': '2024-12-31T07:11:21.127422+00:00'}, {'id': 29, 'comment': 'I learned a lot about the importance of communication and organization within the club.', 'itinerary_id': 'eIO1lSRwFdmgCtvOC3PX', 'user_id': '6huDCteYDyfr2OXenMVVEQjMufs2', 'created_at': '2024-12-31T07:11:21.127422+00:00'}, {'id': 30, 'comment': 'The seminar covered everything I needed to know about managing club records and meetings.', 'itinerary_id': '6AsZlajvoMCNd3M46M7c', 'user_id': 'EU1O3wRX8UfVa7fs0CpYev9eBsc2', 'created_at': '2024-12-31T07:11:2